<a href="https://colab.research.google.com/github/xbadiam/Portfolio/blob/main/Preventive_Maintenance/Nasa_Turbofan_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NASA Turbofan Engine

## Mantenimiento Predictivo en la Industria 4.0: Anticipación de Fallos y Optimización de Maquinaria mediante Machine Learning.

## 1. Introducción

El objetivo de este conjunto de datos es predecir la vida útil remanente (RUL, por sus siglas en inglés) de cada motor en el conjunto de prueba. La RUL equivale al número de vuelos restantes para el motor después del último punto de datos en el conjunto de prueba.






In [2]:
! git clone https://github.com/xbadiam/Portfolio.git

Cloning into 'Portfolio'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 119 (delta 31), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (119/119), 9.87 MiB | 5.27 MiB/s, done.
Resolving deltas: 100% (31/31), done.


## 2. Imports

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## 3. Funciones especiales

## 4. Carga de los datos



In [12]:
# Cargar los datos de entrenamiento
train_data = pd.read_csv("/content/Portfolio/Preventive_Maintenance/train_FD001.txt", sep=' ', header=None)

# Cargar los datos de prueba
test_data = pd.read_csv("/content/Portfolio/Preventive_Maintenance/test_FD001.txt", sep=' ', header=None)

# Cargar la vida útil restante verdadera
truth_data = pd.read_csv("/content/Portfolio/Preventive_Maintenance/RUL_FD001.txt", header=None)


### 4.1. Conjunto de datos

A continuación, se asigna los nombres a las columnas de los tres ficheros. Para ello, se consulta los nombres de las variables facilitados en el entorno web de NASA's Open Data Portal. Cada fila es un instante de los datos tomados durante un único ciclo operativo y cada columna es una variable diferente.

* unit_number: identificador de la unidad (motor)
* time_cycles: time, in cycles
* op_set1: configuración operativa 1
* op_set2: configuración operativa 2
* op_set3: configuración operativa 3
* s_1: medición del sensor 1
* s_2: medición del sensor 1
.......
* s_26 medición del sensor 26

Se cargan tres ficheros de tipo CSVcon los datos de entrenamiento formado por las lecturas de sensores a lo largo d e ciclos operativos completos. Con el fin de dar mejor claridad al alto número de variables en la exploración de los datos, se definen dos variables que permiten separar las variables en dos grupos: datos del motor y los sensores.

In [20]:
#Eliminanos las dos últimas columnas, ya que solo presentan valores NaN
train_data = train_data[train_data.columns[:-2]]
test_data = test_data[test_data.columns[:-2]]

In [21]:
# Define los nombres configuración de los motores y de los sensores separados
unit_settings_names = ['unit_number', 'time_cycles','op_set1', 'op_set2', 'op_set3']
sensor_names = ['s_{}'.format(i) for i in range(1,22)]

# Conjunto de los nombres de las variables
column_names = unit_settings_names + sensor_names

train_data.columns = column_names
test_data.columns = column_names
truth_data.columns = ["RUL"]


In [22]:
train_data.head()


,unit_number,time_cycles,op_set1,op_set2,op_set3,s_1,s_2,s_3,s_4,s_5,...,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20,s_21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
